In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

import deeplabcut


Autosaving every 180 seconds


In [2]:
# READ X,Y and Likelihoods from a presaved CSV file
   
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]

    # load values
    data_array = np.array(data[3:])

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    for idx in range(1,len(labels)-1,3):
        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan

def load_csv_data(data_all):
    traces = np.array(data_all[0])
    #print (traces[0].shape)
    tracesx = []
    tracesy = []
    likelihoods = []

    for k in range(0,len(traces),1):
        tracesx.append(traces[k][:,0])
        tracesy.append(traces[k][:,1])
        likelihoods.append(traces[k][:,2])

    likelihoods = np.array(likelihoods)
    tracesx = np.array(tracesx)
    tracesy = np.array(tracesy)
    print ("tracex: ", tracesx.shape)
    print ("tracey: ", tracesy.shape)
    print ("likelihood: ", likelihoods.shape)
        
    return tracesx, tracesy, likelihoods



In [4]:
# load a csv file; 
# usually a bx.csv file
#csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_filtered.csv'
#csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
#csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
#csv_fname = '/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
csv_fname = '/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_54_23_358257_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx.csv'
data_all = load_csv(csv_fname)
tracesx, tracesy, likelihoods = load_csv_data(data_all)

#traces_all = np.stack((np.stack((tracesx,tracesy)),likelihoods))
print ("DONE Loading traces and likelihoods")

tracex:  (56, 89988)
tracey:  (56, 89988)
likelihood:  (56, 89988)
DONE Loading traces and likelihoods


In [5]:
# MAKE VIDOES + LABELS 

# Robust outlier detection function to crop some of the DLC errors;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

In [6]:
# Save median/single feature trackelts and likelihoods
# initialize start and ends of video to be analyzed
start = 0
end = tracesx.shape[1]

tracesx_mean =[]
tracesy_mean =[]
likelihoods_mean =[]
threshold = 0.01

total_nans = [0]*4
print (total_nans)
for k in range(start,end,1):    
    if k%5000==0:
        print (k, "/", end)
        
    tracesx_mean.append([])
    tracesy_mean.append([])
    # PLOT MEAN FEATURE
    for p in range(0, tracesx.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]

        traces_local = tracesx[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 4)
        x = np.nanmedian(traces_local)
        
        traces_local = tracesy[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 4)
        y = np.nanmedian(traces_local)
        
        tracesx_mean[k].append(x)
        tracesy_mean[k].append(y)
        
        if np.isnan(x) or np.isnan(y):
            total_nans[p//14]+=1

tracesx_mean = np.array(tracesx_mean)

print ("total nans:", total_nans)

print ("DONE Loading mean traces")
np.savez(csv_fname[:-4]+'_single_feature_traces.npz', 
        tracesx=tracesx_mean,
        tracesy=tracesy_mean)

[0, 0, 0, 0]
0 / 89988


/home/madeline/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/madeline/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/madeline/anaconda3/envs/DLC-GPU/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


5000 / 89988
10000 / 89988
15000 / 89988
20000 / 89988
25000 / 89988
30000 / 89988
35000 / 89988
40000 / 89988
45000 / 89988
50000 / 89988
55000 / 89988
60000 / 89988
65000 / 89988
70000 / 89988
75000 / 89988
80000 / 89988
85000 / 89988
total nans: [17072, 25204, 46005, 73587]
DONE Loading mean traces


In [1]:
# filter data
total=[1964, 4534, 5890, 9721]

In [4]:
import numpy as np
print (np.array(total)/16572)

[0.11851315 0.27359401 0.35541878 0.58659184]
